In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [25]:
data = pd.read_csv("C:\\Users\\mahik\\OneDrive\\Desktop\\datasetrestro\\Dataset .csv")  

In [28]:
print(data.isnull().sum())

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64


In [33]:
data.dropna(subset=['Cuisines', 'Rating text'], inplace=True)

In [34]:
print(" Cleaned dataset:")
print(data.head())

 Cleaned dataset:
   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong 

In [35]:
data['features'] = data['Cuisines']  + data['City']+ data['Rating text'] + data['Has Table booking']

In [36]:

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['features'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print (cosine_sim)

[[1.         0.32857524 0.         ... 0.         0.         0.        ]
 [0.32857524 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.75276275]
 [0.         0.         0.         ... 0.         0.75276275 1.        ]]


In [37]:
def recommend_restaurants(restaurant_name, cosine_sim=cosine_sim):
    idx = data[data['Restaurant Name'] == restaurant_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  
    restaurant_indices = [i[0] for i in sim_scores]
    return data['Restaurant Name'].iloc[restaurant_indices]


In [38]:
restaurant_name = "Sambo Kojin"  
recommended_restaurants = recommend_restaurants(restaurant_name)
print("Recommended restaurants for {}: \n{}".format(restaurant_name, recommended_restaurants))

Recommended restaurants for Sambo Kojin: 
0                               Le Petit Souffle
1                               Izakaya Kikufuji
9                                       Locavore
17                                    Balay Dako
6400                             Dimsum Vs Sushi
8       Spiral - Sofitel Philippine Plaza Manila
21                 The Food Hall by Todd English
20                                NIU by Vikings
1169                                         Red
3238                               Moets Oh! Bao
Name: Restaurant Name, dtype: object


In [39]:
user_preferences = {
    'favorite_restaurant': "Nonna's Pasta & Pizzeria",
    'favorite_cuisine': "Italian"
}


favorite_restaurant = user_preferences['favorite_restaurant']
recommended_restaurants = recommend_restaurants(favorite_restaurant)


print("Recommended restaurants based on user preferences:")
print(recommended_restaurants)

Recommended restaurants based on user preferences:
6717                        The Drunk House
2751                           Munch Nation
7863                        Big Yellow Door
7862                               736 A.D.
3337                        The Hudson Cafe
7036                           Cafe Kazbaah
3703             Sakley's The Mountain Cafe
7046                   Young Wild Free Cafe
7041    QRO Gourmeteriia BY DARK HOUSE KAFE
201                                Deorio's
Name: Restaurant Name, dtype: object
